# Анализ базы данных сервиса для чтения книг


## Цели исследования:
1. Изучить информацию о книгах, издательствах, авторах, а также пользовательских обзорах книг.
2. Сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
con = engine
#функция для вызова sql запроса
def select(sql):
    return pd.read_sql(sql,con)

In [3]:
check1='''select * from books limit 10'''
select(check1)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [4]:
check2='''select * from authors limit 10'''
select(check2)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


In [5]:
check3='''select * from publishers limit 10'''
select(check3)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


In [6]:
check4='''select * from ratings limit 10'''
select(check4)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [7]:
check5='''select * from reviews limit 10'''
select(check5)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


In [8]:
query1='''select COUNT(DISTINCT book_id) 
from books 
where publication_date > '2000-01-01'
'''
select(query1) 

,count
0,819


**819 книг вышло после 1 января 2000 года. Всего книг в таблице - 1000.**

In [9]:
query2='''SELECT b.book_id, title, COUNT(DISTINCT re.review_id) AS reviews, AVG(ra.rating) as avg_rating
FROM books b
LEFT JOIN ratings ra ON b.book_id=ra.book_id
LEFT JOIN reviews re ON b.book_id=re.book_id
GROUP BY b.book_id, title
'''
select(query2) 

,book_id,title,reviews,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


**Для каждой книги посчитал количество обзоров и среднюю оценку, даже если где-то в данных пропуск.**

In [10]:
query3='''SELECT p.publisher, COUNT(b.book_id) 
FROM books b
JOIN publishers p on b.publisher_id=p.publisher_id
WHERE b.num_pages>50
GROUP BY p.publisher
ORDER BY COUNT(b.book_id) DESC
LIMIT 1'''
select(query3) 

,publisher,count
0,Penguin Books,42


**Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц.**

In [11]:
query4='''WITH data as (SELECT b.book_id, b.author_id, AVG(ra.rating) as avg_rating
FROM books b
FULL JOIN ratings ra ON b.book_id=ra.book_id
GROUP BY b.book_id, b.author_id
HAVING COUNT(ra.rating_id)>=50
ORDER BY AVG(ra.rating) DESC
LIMIT 1)

SELECT author
FROM authors a
JOIN data d on a.author_id=d.author_id
'''
select(query4) 

,author
0,J.K. Rowling/Mary GrandPré


**J.K. Rowling - автор с самой высокой средней оценкой книг. Mary GrandPré - иллюстратор книг о Гарри Поттере.**

In [17]:
query5='''

SELECT AVG(t.cnt)

FROM 
(SELECT re.username, COUNT(re.review_id) as cnt

FROM reviews re
WHERE re.username IN (
SELECT ra.username
FROM ratings ra 
GROUP BY ra.username
HAVING COUNT(ra.rating_id)>50)
GROUP BY 1) t


'''
select(query5) 


,avg
0,24.333333


**24 - среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

# Общий вывод




**819 книг вышло после 1 января 2000 года. Всего книг в таблице - 1000.**

**Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц.**

**J.K. Rowling - автор с самой высокой средней оценкой книг. Mary GrandPré - иллюстратор книг о Гарри Поттере.**

**24 - среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**